## Taylor Rule Deviation:

$i_t = 2 + \pi_t + a(\pi_t-\pi^*)+b(y_t - y_t^*)$

+ From the formula above we need:

    + Interest Rate
    + Inflation Rate
    + Targetted Inflation Rate
    + Potential Output

### Interest Rates:

Sources: https://iftp.chinamoney.com.cn/english/bmklpr/, https://www.bis.org/statistics/cbpol/cbpol.xlsx

I need to quarterlyize the interest rates